## Imports



In [ ]:
from common_language import _LANGUAGES
import processing as prlib
import extract_features as featurelib

from scipy.fft import fft
from mutagen.wave import WAVE
from parselmouth.praat import call
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa
import librosa.display
import parselmouth
import noisereduce as nr
import os 

##Features processing

###Get data frames

In [ ]:
print(os.getcwd())
_, train_df, test_df, validation_df = prlib.get_clean_dataframes()
train_df.head(), prlib.frame_length

### Features matrix

In [ ]:
type = 'validation'
datasets = [('train', train_df), ('test', test_df), ('validation', validation_df)]

for (type, df) in datasets[2:]:
    attributes_df, df = featurelib.extract_features(type, df.iloc[:100, :])
    #attributes_df.to_csv(f'./data/{type}_preprocessed_data.csv', index=False) # , sep=',', encoding='utf-8', index=False)
    #df.to_csv(f'./data/wav_files_clean/{type}/{type}_data.csv', index=False)

attributes_df

## Features visualisation

### Initialisation

In [ ]:
data = prlib.get_data(path=f'./data/wav_files/validation/{validation_df['paths'][0]}')

### MFCCs visualisation

In [ ]:
normalized_mfccs = prlib.get_Normalized_Mfccs(data)

plt.figure(figsize=(12,6))
librosa.display.specshow(normalized_mfccs, x_axis='time')
plt.colorbar()
plt.title('MFCCs')
plt.tight_layout()
plt.show()

### Spectral measurements visualisation

In [ ]:
specs = prlib.get_spectral_measurements(data)

t = librosa.frames_to_time(range(len(specs[0])), sr=prlib.sample_rate)

# Plotting the Spectral Features
plt.figure(figsize=(12, 4))
plt.plot(t, specs[0], color='red', label='Centroid')
plt.plot(t, specs[1], color='blue', label='Rolloff')
plt.plot(t, specs[2], color='green', label='Bandwidth')

# For spectral flatness, there's no need to convert to dB since it's a ratio and typically small.
#plt.plot(t, spectral_flatness, color='orange', label='Flatness')

# For spectral contrast, it's common to average over the frequency bands since it returns an array of shape (n_bands, n_frames).
#spectral_contrast_avg = np.mean(spectral_contrast, axis=0)
#plt.plot(t, spectral_contrast_avg, color='black', label='Contrast')

plt.xlabel("Time (s)")
plt.ylabel("Spectral Feature Value")
plt.title("Spectral Features Over Time")
plt.legend(loc='best')
plt.show()

### Pitches visualisation

In [ ]:
pitch_track = prlib.get_pitch_sequences(data)
plt.figure(figsize=(12, 6))
plt.plot(pitch_track)
plt.xlabel('Time (frames)')
plt.ylabel('Frequency (Hz)')
plt.title('Pitch Track')
plt.show()

### RMS Visualisation

In [ ]:
rms_energy = prlib.get_rms_energy(data)
frames = range(len(rms_energy[0]))
t = librosa.frames_to_time(frames, sr=prlib.sample_rate)

plt.figure(figsize=(12, 4))
plt.plot(t, rms_energy[0], label='RMS Energy')
plt.xlabel("Time (s)")
plt.ylabel("Energy")
plt.title("RMS Energy Over Time")
plt.legend()
plt.show()